In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from datetime import datetime
import socket
import getpass
import time
import os

In [16]:
def create_gradient_background(ax, cmap=None, resolution=100, alpha=0.7):
    if cmap is None:
        cmap = create_custom_colormap()

    gradient = np.linspace(0, 1, resolution)
    gradient = np.vstack((gradient, gradient))

    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    ax.imshow(gradient, aspect='auto', cmap=cmap, origin='lower', alpha=alpha, extent=[*xlim, *ylim])

def create_custom_colormap():
    from matplotlib.colors import LinearSegmentedColormap
    custom_colors = [(0, 'black'),  (1, '#2b2b5b')]  # Slightly lighter deep purple/grey color
    custom_cmap = LinearSegmentedColormap.from_list('custom_cmap', custom_colors)
    return custom_cmap

def filter_outliers_by_date(df, column='LODGEMENT_DATE', min_year=2000):
    min_date = pd.Timestamp(year=min_year, month=1, day=1)
    return df[df[column] >= min_date]

def remove_nan_dates(df, column='LODGEMENT_DATE'):
    return df[df[column].notna()]


def plot_average_energy_efficiency(df):
    df['LODGEMENT_DATE'] = pd.to_datetime(df['LODGEMENT_DATE'])
    df['YEAR'] = df['LODGEMENT_DATE'].dt.year
    df['MONTH'] = df['LODGEMENT_DATE'].dt.month
    
    
    avg_energy_efficiency = df.groupby(['YEAR', 'MONTH'])['CURRENT_ENERGY_EFFICIENCY'].mean().reset_index()
    
    # Calculate the percentiles for each month
    percentiles = [0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]
    monthly_percentiles = df.groupby(['YEAR', 'MONTH'])['CURRENT_ENERGY_EFFICIENCY'].quantile(percentiles).unstack().reset_index()
    
    # Create a 'DATE' column using the 'YEAR' and 'MONTH' columns
    avg_energy_efficiency['DATE'] = pd.to_datetime(avg_energy_efficiency[['YEAR', 'MONTH']].assign(day=1))
    monthly_percentiles['DATE'] = pd.to_datetime(monthly_percentiles[['YEAR', 'MONTH']].assign(day=1))
    
    with plt.style.context('dark_background'):
        fig, ax = plt.subplots(figsize=(12, 6))

        ax.plot(avg_energy_efficiency['DATE'], avg_energy_efficiency['CURRENT_ENERGY_EFFICIENCY'], color='lime', label='Avg Energy Efficiency')

        # Plot the percentiles for each month
        for p in percentiles:
            ax.plot(monthly_percentiles['DATE'], monthly_percentiles[p], linestyle='--', alpha=0.6, label=f'{p * 100}%')

        create_gradient_background(ax)
        
        # Set the x-axis to display years and months
        ax.xaxis.set_major_locator(mdates.YearLocator())
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
                
        plt.xlabel('Date', color='white', fontsize=12)
        plt.ylabel('Average Energy Efficiency', color='white', fontsize=12)
        plt.title('Average Energy Efficiency Over Time', color='white', fontsize=16, pad=20)
        ax.tick_params(axis='both', which='major', labelsize=10, colors='white', labelleft=True)
        ax.tick_params(axis='x', which='minor', labelsize=8, colors='white', labelleft=True)
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        ax.text(1, 1.01, f"Generated: {timestamp}", fontsize=10, ha='right', transform=ax.transAxes, color='white')

        plt.legend(loc='upper left', fontsize=10)

        # Calculate the total number of reports lodged per year
        reports_per_year = df.groupby(['YEAR'])['LODGEMENT_DATE'].count().reset_index()
        reports_per_year.columns = ['Year', 'Total Reports']
        reports_per_year['Total Reports'] = reports_per_year['Total Reports'].apply(lambda x: f'{x:,}')

        # Add table to the right of the main graph
        table = plt.table(cellText=reports_per_year.values,
                        colLabels=reports_per_year.columns,
                        loc='right',
                        cellLoc='center',
                        colColours=['#404040'] * 2,
                        colWidths=[0.15, 0.15],  # Increase column widths
                        cellColours=[['#606060', '#606060']] * len(reports_per_year),
                        edges='closed')
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1, 1.5)  # Increase table height

        # Set the horizontal alignment for the 'Total Reports' column
         # Set the horizontal alignment for the 'Total Reports' column
        for i in range(len(reports_per_year) + 1):  # Add 1 to the range to include the last row
            table[i, 1].set_text_props(ha='right')

        plt.subplots_adjust(right=0.7)  # Move the table further to the right

          # Add machine/user account name in the bottom right corner
        machine_user = getpass.getuser()
        machine_name = socket.gethostname()
        ax.text(1, -0.1, f"Generated by: {machine_user}@{machine_name}", fontsize=10, ha='right', transform=ax.transAxes, color='white')  # Move the user name down

        # Calculate and display the time taken to generate the report
        elapsed_time = time.time() - start_time
        minutes, seconds = divmod(elapsed_time, 60)
        ax.text(1, -0.15, f"Time taken: {int(minutes)}m {int(seconds)}s", fontsize=10, ha='right', transform=ax.transAxes, color='white')  # Add the time taken below the user name

        # Create a separate table for the latest lodgement date
        latest_lodgement = df['LODGEMENT_DATE'].max()
        latest_lodgement_formatted = latest_lodgement.strftime('%Y-%m-%d')
        latest_lodgement_data = pd.DataFrame({'Latest Lodgement': [latest_lodgement_formatted]})

        latest_lodgement_table = plt.table(cellText=latest_lodgement_data.values,
                                        colLabels=latest_lodgement_data.columns,
                                        loc='lower right',
                                        bbox=[1.05, -0.28, 0.25, 0.1],  # Adjust the position and width of the table
                                        cellLoc='center',
                                        colColours=['#404040'],
                                        cellColours=[['#606060']],
                                        edges='closed')
        latest_lodgement_table.auto_set_font_size(False)
        latest_lodgement_table.set_fontsize(10)

        # ... (The rest of the code remains unchanged)

        plt.savefig('energy_efficiency_graph.png', dpi=300, bbox_inches='tight')
        plt.show()

def plot_correlation_matrix(df):
    # Bucket the floor area
    bins = [0, 500, 1000, 1500, 2000, np.inf]
    labels = ['<500', '500-1000', '1000-1500', '1500-2000', '>2000']
    df['FLOOR_AREA_BUCKET'] = pd.cut(df['TOTAL_FLOOR_AREA'], bins=bins, labels=labels)

    # Calculate the mean floor area for each postcode
    postcode_floor_area = df.groupby('Postcode')['TOTAL_FLOOR_AREA'].mean().reset_index()

    # Merge the dataframes on Postcode
    merged_df = df.merge(postcode_floor_area, on='Postcode', suffixes=('', '_mean'))

    # Calculate the correlation matrix
    correlation_matrix = merged_df[['CURRENT_ENERGY_EFFICIENCY', 'TOTAL_FLOOR_AREA_mean']].corr()

    # Plot the correlation matrix
    plt.figure(figsize=(6, 6))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', square=True)
    plt.title('Correlation Matrix')
    plt.show()
